In [1]:
import os
import pandas as pd

In [2]:
%pwd

'/Users/harkiratchahal/Desktop/Coding/Tutorials/mlops_2/research'

In [3]:
os.chdir('../')

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
os.environ['MLFLOW_TRACKING_URI']='https://dagshub.com/iamharkirat/campus_recruitment.mlflow' 
os.environ['MLFLOW_TRACKING_USERNAME']='iamharkirat' 
os.environ['MLFLOW_TRACKING_PASSWORD']='3674ba2c6ee8b99eefb716575a7781d71ccbe637' 

In [13]:
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [14]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [15]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.RandomForestClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/iamharkirat/campus_recruitment.mlflow",
        )

        return model_evaluation_config


In [23]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
from mlProject import logger
import joblib

In [24]:
class ModelEvaluation:
    def __init__(self, config):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        accuracy = accuracy_score(actual, pred)
        logger.info(f"Evaluation Metrics - RMSE: {rmse}, MAE: {mae}, R2 Score: {r2}, Accuracy: {accuracy}")
        return rmse, mae, r2, accuracy

    def save_metrics_locally(self, scores):
        # Ensure you have a save_json function or use an appropriate method to save the scores as JSON.
        save_json(path=Path(self.config.metric_file_name), data=scores)

    def log_into_mlflow(self):
        logger.info("Model evaluation started.")
        logger.info("Starting MLflow logging...")

        # Load test data and model
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        # Split data into features and target
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)

            # Evaluate metrics
            rmse, mae, r2, accuracy = self.eval_metrics(test_y, predicted_qualities)

            # Save metrics locally and log parameters and metrics to MLflow
            scores = {"rmse": rmse, "mae": mae, "r2": r2, "accuracy": accuracy}
            self.save_metrics_locally(scores)
            logger.info("Metrics saved locally.")
            
            logger.info("Logging parameters and metrics to MLflow.")
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(scores)

            # Log model to MLflow
            logger.info("Logging model to MLflow.")
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="RandomForestClassifier")
                logger.info("Model registered in MLflow.")
            else:
                mlflow.sklearn.log_model(model, "model")
                logger.info("Model logged to MLflow.")
        
        logger.info("Model evaluation completed.")

In [25]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e


[2023-10-03 17:23:03,390: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-03 17:23:03,393: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-03 17:23:03,395: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-03 17:23:03,395: INFO: common: created directory at: artifacts]
[2023-10-03 17:23:03,396: INFO: common: created directory at: artifacts/model_evaluation]
[2023-10-03 17:23:03,396: INFO: 178573336: Model evaluation started.]
[2023-10-03 17:23:03,397: INFO: 178573336: Starting MLflow logging...]
[2023-10-03 17:23:08,972: INFO: 178573336: Evaluation Metrics - RMSE: 0.0, MAE: 0.0, R2 Score: 1.0, Accuracy: 1.0]
[2023-10-03 17:23:08,973: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]
[2023-10-03 17:23:08,974: INFO: 178573336: Metrics saved locally.]
[2023-10-03 17:23:08,974: INFO: 178573336: Logging parameters and metrics to MLflow.]
[2023-10-03 17:23:10,393: INFO: 178573336: Logging model to MLflow.]

Registered model 'RandomForestClassifier' already exists. Creating a new version of this model...
2023/10/03 17:23:17 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: RandomForestClassifier, version 2


[2023-10-03 17:23:17,141: INFO: 178573336: Model registered in MLflow.]
[2023-10-03 17:23:17,309: INFO: 178573336: Model evaluation completed.]


Created version '2' of model 'RandomForestClassifier'.
